### Exploratory Data Analysis with AirBnb dataset

In [244]:
import pandas as pd

Read dataset containing accomodations attributes

In [245]:
listings = pd.read_csv('data/listings.csv.gz')

/home/off-andre/miniconda3/envs/descritivo/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (57,58,88) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [246]:
listings.head(3)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,requires_license,jurisdiction_names,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17878,https://www.airbnb.com/rooms/17878,20200722063547,2020-07-22,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. Please contact ...,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/im/pictures/65320518/3...,68997,https://www.airbnb.com/users/show/68997,...,f,NaN,f,moderate,f,f,1,0,0,2.07
1,21280,https://www.airbnb.com/rooms/21280,20200722063547,2020-07-22,Renovated Modern Apt. Near Beach,Immaculately renovated top-floor apartment ove...,This is the best neighborhood in Zona Sul. Fo...,https://a0.muscache.com/im/pictures/60851312/b...,81163,https://www.airbnb.com/users/show/81163,...,f,NaN,f,strict_14_with_grace_period,f,f,1,0,0,1.20
2,25026,https://www.airbnb.com/rooms/25026,20200722063547,2020-07-22,Beautiful Modern Decorated Studio in Copa,"Our apartment is a little gem, everyone loves ...",Copacabana is a lively neighborhood and the ap...,https://a0.muscache.com/im/pictures/3003965/68...,102840,https://www.airbnb.com/users/show/102840,...,f,NaN,f,strict_14_with_grace_period,t,t,3,0,0,1.93


Clean data and keep only useful columns

In [247]:
useful_columns =['host_is_superhost', 'neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'guests_included', 'review_scores_rating']

In [248]:
listings = listings[useful_columns]

Read dataset containing reviews

In [249]:
reviews = pd.read_csv('data/reviews.csv.gz')

In [250]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
1,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
2,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
3,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r\n...
4,17878,147594,2010-12-01,219338,David,The apt was nice and in a great location only ...


#### Data Analysis

In [251]:
listings.head()

,id,name,description,neighborhood_overview,host_since,host_is_superhost,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,summary,space,transit,access,weekly_price,monthly_price,cleaning_fee,guests_included,review_scores_rating
0,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. Please contact ...,This is the one of the bests spots in Rio. Bec...,2010-01-08,t,Copacabana,Condominium,Entire home/apt,5,1.0,Discounts for long term stays. Please contact ...,"- Beautiful, sunny 2 bedroom (90 square meters...",Excellent location. Close to all major attract...,The entire apartment is yours. It's like your ...,NaN,NaN,$250.00,2.0,93.0
1,21280,Renovated Modern Apt. Near Beach,Immaculately renovated top-floor apartment ove...,This is the best neighborhood in Zona Sul. Fo...,2010-02-14,f,Ipanema,Apartment,Entire home/apt,6,2.0,Immaculately renovated top-floor apartment ove...,Immaculately renovated top-floor apartment in ...,The new metro station is just a few steps away...,"This is an older ""Art Deco"" style building, so...","$4,917.00","$17,357.00",$263.00,6.0,97.0
2,25026,Beautiful Modern Decorated Studio in Copa,"Our apartment is a little gem, everyone loves ...",Copacabana is a lively neighborhood and the ap...,2010-04-03,f,Copacabana,Apartment,Entire home/apt,2,1.0,"Our apartment is a little gem, everyone loves ...",This newly renovated studio (last renovations ...,At night we recommend you to take taxis only. ...,"internet wi-fi, cable tv, air cond, ceiling fa...",NaN,NaN,$250.00,2.0,94.0
3,31560,NICE & COZY 1BDR - IPANEMA BEACH,This nice and clean 1 bedroom apartment is loc...,Die Nachbarschaft von Ipanema ist super lebend...,2010-05-31,t,Ipanema,Apartment,Entire home/apt,3,1.0,This nice and clean 1 bedroom apartment is loc...,This nice and clean 1 bedroom apartment is loc...,"Bus, U-Bahn, Taxi und Leihfahrräder in der Nähe.","Die Urlauber dürfen das Badezimmer benutzen, d...",NaN,NaN,$105.00,2.0,96.0
4,35636,Cosy flat close to Ipanema beach,This cosy apartment is just a few steps away ...,The apartment street is very quiet and safe ....,2010-06-27,f,Ipanema,Apartment,Entire home/apt,2,1.5,This cosy apartment is just a few steps away ...,"The location is extremely convenient, safe and...",Metro stop just 5 blocks from our place. Buses...,NaN,NaN,"$11,829.00",$211.00,2.0,94.0


In [253]:
complex_feature_cols = ['is_superhost', 
                       'accommodates_1_5', 'accommodates_5_10', 'accomodates_10_15', 'accomodates_15_plus',
                       'bath_1', 'bath_2', 'bath_2_5', 'bath_5_plus', 
                       'guests_included_1_2', 'guests_included_2_6', 'guests_included_6_plus'
                      'rating_0_60', 'rating_60_70','rating_70_80', 'rating_80_90', 'rating_90_100']

In [254]:
for i in useful_columns:
    print(i)
    print('Qtd ' + str(len(listings[i].unique())))
    print(listings[i].value_counts())
    print('----------------------------------------------')

#### Transactional format

In [255]:
feature_cols = ['id'] + complex_feature_cols + list(listings['neighbourhood_cleansed'].unique()) + list(listings['room_type'].unique()) + list(listings['property_type'].unique())

In [256]:
len(feature_cols)

227

In [257]:
df_t = pd.DataFrame(columns=feature_cols)

In [258]:
df_t

,id,is_superhost,accommodates_1_5,accommodates_5_10,accomodates_10_15,accomodates_15_plus,bath_1,bath_2,bath_2_5,bath_5_plus,...,Entire serviced apartment,Houseboat,Vacation home,Casa particular (Cuba),Hut,Camper/RV,Dorm,Shared room in serviced apartment,Island,Barn


In [259]:
# create empty template for rows
def initialize_rows():
    t_row = {}
    for c in df_t.columns:
        t_row[c] = 0
    return t_row

In [260]:
listings = listings[['id'] + useful_columns]
listings

,id,host_is_superhost,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,guests_included,review_scores_rating
0,17878,t,Copacabana,Condominium,Entire home/apt,5,1.0,2.0,93.0
1,21280,f,Ipanema,Apartment,Entire home/apt,6,2.0,6.0,97.0
2,25026,f,Copacabana,Apartment,Entire home/apt,2,1.0,2.0,94.0
3,31560,t,Ipanema,Apartment,Entire home/apt,3,1.0,2.0,96.0
4,35636,f,Ipanema,Apartment,Entire home/apt,2,1.5,2.0,94.0
...,...,...,...,...,...,...,...,...,...
35250,44337570,f,Leblon,Apartment,Entire home/apt,4,2.0,1.0,NaN
35251,44337678,f,Copacabana,Apartment,Entire home/apt,4,1.0,1.0,NaN
35252,44338103,f,Freguesia (Jacarepaguá),Apartment,Private room,2,1.5,1.0,NaN
35253,44338273,f,Copacabana,Apartment,Shared room,2,1.5,1.0,NaN


In [261]:
for index, listing in listings.iterrows():
    t_row = initialize_rows()
    t_row['id'] = listing['id']
    t_row['is_superhost'] = 1 if listing['host_is_superhost'] == 't' else 0
    t_row[listing['neighbourhood_cleansed']] == 1
    t_row[listing['property_type']] == 1
    t_row[listing['room_type']] == 1
    
    # accommodations
    if listing['accommodates'] <= 5:
        t_row['accommodates_1_5'] = 1
    elif listing['accommodates'] <= 10:
        t_row['accommodates_5_10'] = 1
    elif listing['accommodates'] <= 15:
        t_row['accommodates_10_15'] = 1
    else:
        t_row['accommodates_15_plus'] = 1
    
    # bathrooms
    if listing['bathrooms'] == 1:
        t_row['bath_1'] = 1
    elif listing['bathrooms'] == 2:
        t_row['bath_2'] = 1
    elif listing['bathrooms'] <= 5:
        t_row['bath_2_5'] = 1
    else:
        t_row['bath_5_plus'] = 1
         
    # guests_included
    if listing['guests_included'] <=  2:
        t_row['guests_included_1_2'] = 1
    elif listing['guests_included'] <= 6:
        t_row['guests_included_2_6'] = 1
    else:
        t_row['guests_included_6_plus'] = 1
        
    # rating rating_0_60', 'rating_60_70','rating_70_80', 'rating_90_100
    if listing['review_scores_rating'] <= 60:
        t_row['rating_0_60'] = 1
    elif listing['review_scores_rating'] <= 70:
        t_row['rating_60_70'] = 1
    elif listing['review_scores_rating'] <= 80:
        t_row['rating_70_80'] = 1
    elif listing['review_scores_rating'] <= 90:
        t_row['rating_80_90'] = 1
    else:
        t_row['rating_90_100'] = 1
    
    
    
    df_t = df_t.append(t_row, ignore_index=True)

In [262]:
df_t

,id,is_superhost,accommodates_1_5,accommodates_5_10,accomodates_10_15,accomodates_15_plus,bath_1,bath_2,bath_2_5,bath_5_plus,...,Hut,Camper/RV,Dorm,Shared room in serviced apartment,Island,Barn,accommodates_10_15,guests_included_6_plus,accommodates_15_plus,rating_0_60
0,17878,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
1,21280,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,25026,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3,31560,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
4,35636,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35250,44337570,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
35251,44337678,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
35252,44338103,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
35253,44338273,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


In [265]:
df_t.to_csv('transactional_db.csv')

In [268]:
df_t

,id,is_superhost,accommodates_1_5,accommodates_5_10,accomodates_10_15,accomodates_15_plus,bath_1,bath_2,bath_2_5,bath_5_plus,...,Hut,Camper/RV,Dorm,Shared room in serviced apartment,Island,Barn,accommodates_10_15,guests_included_6_plus,accommodates_15_plus,rating_0_60
0,17878,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
1,21280,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,25026,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3,31560,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
4,35636,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35250,44337570,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
35251,44337678,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
35252,44338103,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
35253,44338273,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


In [269]:
df_t = df_t.fillna(0)

In [270]:
df_t

,id,is_superhost,accommodates_1_5,accommodates_5_10,accomodates_10_15,accomodates_15_plus,bath_1,bath_2,bath_2_5,bath_5_plus,...,Hut,Camper/RV,Dorm,Shared room in serviced apartment,Island,Barn,accommodates_10_15,guests_included_6_plus,accommodates_15_plus,rating_0_60
0,17878,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,21280,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2,25026,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
3,31560,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
4,35636,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35250,44337570,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
35251,44337678,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
35252,44338103,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
35253,44338273,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


### Data Mining Algorithms

In [286]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

In [271]:
df_t = df_t.set_index('id')
df_t

,is_superhost,accommodates_1_5,accommodates_5_10,accomodates_10_15,accomodates_15_plus,bath_1,bath_2,bath_2_5,bath_5_plus,guests_included_1_2,...,Hut,Camper/RV,Dorm,Shared room in serviced apartment,Island,Barn,accommodates_10_15,guests_included_6_plus,accommodates_15_plus,rating_0_60
id,,,,,,,,,,,,,,,,,,,,,
17878,1,1,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
21280,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
25026,0,1,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
31560,1,1,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
35636,0,1,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44337570,0,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
44337678,0,1,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
44338103,0,1,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


In [279]:
apriori(df_t, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.751666,(accommodates_1_5)
1,0.844050,(guests_included_1_2)
2,0.896043,(rating_90_100)
3,0.669494,"(guests_included_1_2, accommodates_1_5)"
4,0.672330,"(accommodates_1_5, rating_90_100)"
5,0.762814,"(guests_included_1_2, rating_90_100)"
6,0.601900,"(guests_included_1_2, accommodates_1_5, rating..."


In [284]:
frequent_itemsets = fpgrowth(df_t, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.896043,(rating_90_100)
1,0.844050,(guests_included_1_2)
2,0.751666,(accommodates_1_5)
3,0.762814,"(guests_included_1_2, rating_90_100)"
4,0.672330,"(accommodates_1_5, rating_90_100)"
5,0.669494,"(guests_included_1_2, accommodates_1_5)"
6,0.601900,"(guests_included_1_2, accommodates_1_5, rating..."


In [285]:
frequent_itemsets = fpgrowth(df_t, min_support=0.6, use_colnames=True)

In [287]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(guests_included_1_2),(rating_90_100),0.844050,0.896043,0.762814,0.903754,1.008605,0.006508,1.080113
1,(rating_90_100),(guests_included_1_2),0.896043,0.844050,0.762814,0.851314,1.008605,0.006508,1.048849
2,(accommodates_1_5),(rating_90_100),0.751666,0.896043,0.672330,0.894453,0.998225,-0.001195,0.984933
3,(rating_90_100),(accommodates_1_5),0.896043,0.751666,0.672330,0.750332,0.998225,-0.001195,0.994657
4,(guests_included_1_2),(accommodates_1_5),0.844050,0.751666,0.669494,0.793192,1.055244,0.035049,1.200790
5,(accommodates_1_5),(guests_included_1_2),0.751666,0.844050,0.669494,0.890679,1.055244,0.035049,1.426532
6,"(guests_included_1_2, accommodates_1_5)",(rating_90_100),0.669494,0.896043,0.601900,0.899038,1.003343,0.002005,1.029666
7,"(guests_included_1_2, rating_90_100)",(accommodates_1_5),0.762814,0.751666,0.601900,0.789053,1.049738,0.028519,1.177232
8,"(accommodates_1_5, rating_90_100)",(guests_included_1_2),0.672330,0.844050,0.601900,0.895245,1.060654,0.034420,1.488712
9,(guests_included_1_2),"(accommodates_1_5, rating_90_100)",0.844050,0.672330,0.601900,0.713110,1.060654,0.034420,1.142143
